In [15]:
import mlflow
import os
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient

# Step 2: Set up the MLflow tracking server
load_dotenv()
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))

In [16]:
mlflow.set_experiment("EXP_5-ALL_ML_hpt_7000")

<Experiment: artifact_location='s3://apoorv-mlfow-bucket/602812102375773083', creation_time=1758872187297, experiment_id='602812102375773083', last_update_time=1758872187297, lifecycle_stage='active', name='EXP_5-ALL_ML_hpt_7000', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [17]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from imblearn.over_sampling import ADASYN

from sklearn.model_selection import StratifiedShuffleSplit

In [18]:
df = pd.read_csv('D:\datascience\campusx\PROJECTS\Second_project\cleaned_data.csv',index_col=0).dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [11]:
df['category'].value_counts()


category
 1    15770
 0    12644
-1     8248
Name: count, dtype: int64

In [12]:
import pandas as pd
y_test.value_counts()


category
1    3154
0    2529
Name: count, dtype: int64

In [ ]:
# Step 1: Remap class labels [-1,0,1] -> [2,0,1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# ----------------------------
# Step 2: Stratified Train-Test split
# ----------------------------
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(df['clean_comment'], df['category']):
    X_train, X_test = df['clean_comment'].iloc[train_index], df['clean_comment'].iloc[test_index]
    y_train, y_test = df['category'].iloc[train_index], df['category'].iloc[test_index]

print("Train class distribution:\n", y_train.value_counts())
print("Test class distribution:\n", y_test.value_counts())

# ----------------------------
# Step 3: TF-IDF Vectorization
# ----------------------------
ngram_range = (1, 3)
max_features = 7000

vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# ----------------------------
# Step 4: Handle class imbalance with ADASYN
# ----------------------------
adasyn = ADASYN(random_state=42)
X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)

# ----------------------------
# Step 5: MLflow logging function
# ----------------------------
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, best_params):
    with mlflow.start_run():
        # Log run info
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYN_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log best hyperparameters
        mlflow.log_params(best_params)

        # Log algorithm name
        mlflow.log_param("algo_name", model_name)

        # Train the model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report for all 3 classes
        classification_rep = classification_report(
            y_test,
            y_pred,
            labels=[0, 1, 2],  # Force all classes to appear
            output_dict=True
        )
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

# ----------------------------
# Step 6: Optuna objective function for XGBoost
# ----------------------------
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42
    )
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    return accuracy_score(y_test, y_pred)

# ----------------------------
# Step 7: Run Optuna experiment and log best model + parameters
# ----------------------------
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Best parameters
    best_params = study.best_params

    # Create best model with best parameters
    best_model = XGBClassifier(
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        max_depth=best_params['max_depth'],
        random_state=42
    )

    # Log everything to MLflow
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test, best_params)

# ----------------------------
# Step 8: Run the experiment
# ----------------------------
run_optuna_experiment()

Train class distribution:
 category
1    12616
0    10115
2     6598
Name: count, dtype: int64
Test class distribution:
 category
1    3154
0    2529
2    1650
Name: count, dtype: int64


[I 2025-09-26 13:32:36,866] A new study created in memory with name: no-name-d014071c-a361-438f-a102-28b6ff0da7fb
[I 2025-09-26 13:32:47,801] Trial 0 finished with value: 0.5713896086185736 and parameters: {'n_estimators': 62, 'learning_rate': 0.00025425453149673647, 'max_depth': 7}. Best is trial 0 with value: 0.5713896086185736.
[I 2025-09-26 13:33:43,316] Trial 1 finished with value: 0.5994817946270286 and parameters: {'n_estimators': 288, 'learning_rate': 0.0005455342861891712, 'max_depth': 8}. Best is trial 1 with value: 0.5994817946270286.
[I 2025-09-26 13:33:55,400] Trial 2 finished with value: 0.5400245465702986 and parameters: {'n_estimators': 123, 'learning_rate': 0.0007749490082289659, 'max_depth': 4}. Best is trial 1 with value: 0.5994817946270286.
[I 2025-09-26 13:34:57,723] Trial 3 finished with value: 0.6204827492158734 and parameters: {'n_estimators': 288, 'learning_rate': 0.0010525750884175942, 'max_depth': 9}. Best is trial 3 with value: 0.6204827492158734.
[I 2025-09

🏃 View run XGBoost_ADASYN_TFIDF_Trigrams at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/602812102375773083/runs/acc6182423634159a3a128f0bacc1749
🧪 View experiment at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/602812102375773083


In [11]:
import boto3

s3 = boto3.client("s3", region_name="eu-north-1")
response = s3.list_buckets()
print([bucket['Name'] for bucket in response['Buckets']])

['apoorv-mlfow-bucket']


In [10]:
model_uri="http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/650043532633511350/models/m-705c883531e64a6fbae8f09999170671"
loaded_model = mlflow.sklearn.load_model(model_uri)

ValueError: not enough values to unpack (expected 2, got 1)

In [12]:
import mlflow.sklearn

# Your actual run ID
run_id = "1d05c19a188945adafb55c17d8f950c0"

# Artifact path — must match what you used in log_mlflow
# In your code it was: mlflow.sklearn.log_model(model, f"{model_name}_model")
# So the folder should be "XGBoost_model"
model_uri = f"runs:/{run_id}/XGBoost_model"

In [13]:
loaded_model = mlflow.sklearn.load_model(model_uri)

# Extract the parameters used by this trained model
print("Best parameters used in this model:")
for k, v in loaded_model.get_params().items():
    print(f"{k}: {v}")

Best parameters used in this model:
objective: multi:softprob
base_score: None
booster: None
callbacks: None
colsample_bylevel: None
colsample_bynode: None
colsample_bytree: None
device: None
early_stopping_rounds: None
enable_categorical: False
eval_metric: None
feature_types: None
feature_weights: None
gamma: None
grow_policy: None
importance_type: None
interaction_constraints: None
learning_rate: 0.09087294404864565
max_bin: None
max_cat_threshold: None
max_cat_to_onehot: None
max_delta_step: None
max_depth: 10
max_leaves: None
min_child_weight: None
missing: nan
monotone_constraints: None
multi_strategy: None
n_estimators: 264
n_jobs: None
num_parallel_tree: None
random_state: 42
reg_alpha: None
reg_lambda: None
sampling_method: None
scale_pos_weight: None
subsample: None
tree_method: None
validate_parameters: None
verbosity: None


In [15]:
best_params = {
    "learning_rate": loaded_model.get_params()["learning_rate"],
    "max_depth": loaded_model.get_params()["max_depth"],
    "n_estimators": loaded_model.get_params()["n_estimators"],
    "random_state": loaded_model.get_params()["random_state"],
}

print("Best tuned parameters:", best_params)

Best tuned parameters: {'learning_rate': 0.09087294404864565, 'max_depth': 10, 'n_estimators': 264, 'random_state': 42}
